# Data analytics coursework 

In [2]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the financial_phrasebank dataset
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
from scipy.sparse import hstack
import nltk as nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
train_files = [
    'data/FiQA_ABSA_task1/task1_headline_ABSA_train.json',
    'data/FiQA_ABSA_task1/task1_post_ABSA_train.json'
]

In [4]:
## Loading data from JSON
import json

def load_fiqa_sa_from_json(json_files):
    train_text = []
    train_labels = []

    for file in json_files:
        with open(file, 'r', encoding='UTF-8') as handle:
            dataf = json.load(handle)

        dataf_text = [dataf[k]["sentence"] for k in dataf.keys()]
        # print(len(dataf_text))
        train_text.extend(dataf_text)

        dataf_labels = [float(dataf[k]["info"][0]["sentiment_score"]) for k in dataf.keys()]
        # print(len(dataf_labels))
        train_labels.extend(dataf_labels)

    train_text = np.array(train_text)
    train_labels = np.array(train_labels)
    
    return train_text, train_labels


def threshold_scores(scores):
    """
    Convert sentiment scores to discrete labels.
    0 = negative.
    1 = neutral.
    2 = positive.
    """
    labels = []
    for score in scores:
        if score < -0.2:
            labels.append(0)
        elif score > 0.2:
            labels.append(2)
        else:
            labels.append(1)
            
    return np.array(labels)


all_text, all_labels = load_fiqa_sa_from_json(train_files)
    
print(f'Number of instances: {len(all_text)}')
print(f'Number of labels: {len(all_labels)}')

all_labels = threshold_scores(all_labels)
print(f'Number of negative labels: {np.sum(all_labels==0)}')
print(f'Number of neutral labels: {np.sum(all_labels==1)}')
print(f'Number of positive labels: {np.sum(all_labels==2)}')

Number of instances: 1111
Number of labels: 1111
Number of negative labels: 310
Number of neutral labels: 195
Number of positive labels: 606


In [5]:
from sklearn.model_selection import train_test_split

# Split test data from training data
train_documents, test_documents, train_labels, test_labels = train_test_split(
    all_text, 
    all_labels, 
    test_size=0.2, 
    stratify=all_labels  # make sure the same proportion of labels is in the test set and training set
)

# Split validation data from training data
train_documents, val_documents, train_labels, val_labels = train_test_split(
    train_documents, 
    train_labels, 
    test_size=0.15, 
    stratify=train_labels  # make sure the same proportion of labels is in the test set and training set
)

print(f'Number of training instances = {len(train_documents)}')
print(f'Number of validation instances = {len(val_documents)}')
print(f'Number of test instances = {len(test_documents)}')


Number of training instances = 754
Number of validation instances = 134
Number of test instances = 223


In [6]:
print(f'What does one instance look like from the training set? \n\n{train_documents[234]}')
print(f'...and here is its corresponding label \n\n{train_labels[234]}')

What does one instance look like from the training set? 

Kingfisher takeover of Mr Bricolage could hit a brick wall
...and here is its corresponding label 

0


# Preprocessing using Lemmatization

We create a lemmatizer token that reduces words to their root forms. This reduces the vocabulary size.


In [7]:
class LemmaTokenizer(object):
    
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        
    def __call__(self, tweets):
        return [self.wnl.lemmatize(self.wnl.lemmatize(self.wnl.lemmatize(tok, pos='n'), pos='v'), pos='a') for tok in word_tokenize(tweets)]
    

# Using bi-grams + unigrams as feature

In [51]:
vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(), ngram_range=(1,2))
vectorizer.fit(train_documents)
X_train = vectorizer.transform(train_documents)
X_val = vectorizer.transform(val_documents)
X_test_documents = vectorizer.transform(test_documents)
# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

C:\Users\adnan\anaconda3\envs\data_analytics\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['$', 'amzn', 'reject', 'that', '604', 'to', 'the', 'penny', '.', 'weak', 'open', 'and', 'it', 'go', 'r/g', 'within', 'first', '30', 'min', ',']


In [52]:
print(f'Vocabulary size: {len(vectorizer.vocabulary_)}')
vocabulary = vectorizer.vocabulary_

Vocabulary size: 11282


## Fitting to model

In [53]:
classifier = LogisticRegression()
classifier.fit(X_train, train_labels)
y_test_pred = classifier.predict(X_val)
print(classification_report(val_labels, y_test_pred))

              precision    recall  f1-score   support

           0       0.65      0.54      0.59        37
           1       1.00      0.12      0.22        24
           2       0.68      0.93      0.79        73

    accuracy                           0.68       134
   macro avg       0.78      0.53      0.53       134
weighted avg       0.73      0.68      0.63       134



# Using POS-NEG lexicon features

In [54]:

analyser = SentimentIntensityAnalyzer()

In [55]:
vocabulary = vectorizer.vocabulary_

lex_pos_scores = np.zeros((1, len(vocabulary)))
lex_neg_scores = np.zeros((1, len(vocabulary)))

for i, term in enumerate(vocabulary):
    if term in analyser.lexicon and analyser.lexicon[term] > 0:
        lex_pos_scores[0, i] = 1
    elif term in analyser.lexicon and analyser.lexicon[term] < 0:
        lex_neg_scores[0, i] = 1

In [56]:
# This enables u to get the total positive and total negative counts for each set:
lex_pos_train = np.sum(X_train.multiply(lex_pos_scores), axis=1)
lex_pos_val = np.sum(X_val.multiply(lex_pos_scores), axis=1)
lex_pos_test = np.sum(X_test_documents.multiply(lex_pos_scores), axis=1)

lex_neg_train = np.sum(X_train.multiply(lex_neg_scores), axis=1)
lex_neg_val = np.sum(X_val.multiply(lex_neg_scores), axis=1)

lex_neg_test = np.sum(X_test_documents.multiply(lex_neg_scores), axis=1)

In [58]:
X_train = hstack((X_train, lex_pos_train, lex_neg_train))
X_val = hstack((X_val, lex_pos_val, lex_neg_val))
X_test_documents = hstack((X_test_documents, lex_pos_test, lex_neg_test))

## refitting model

In [15]:

classifier = LogisticRegression()
classifier.fit(X_train, train_labels)
y_test_pred = classifier.predict(X_val)
# Checking performance
print(classification_report(val_labels, y_test_pred))

              precision    recall  f1-score   support

           0       0.65      0.54      0.59        37
           1       1.00      0.12      0.22        24
           2       0.67      0.92      0.77        73

    accuracy                           0.67       134
   macro avg       0.77      0.53      0.53       134
weighted avg       0.72      0.67      0.62       134



In [16]:
# Key part is investigating the errors, so let's do that:
error_indexes = y_test_pred != val_labels  # compare predictions to gold labels

# get the text of tweets where the classifier made an error:
tweets_err = np.array(val_documents)[error_indexes]

# WRITE YOUR CODE HERE
pred_err = y_test_pred[error_indexes]
gold_err = np.array(val_labels)[error_indexes]

for i in range(10):  # just print the first ten
    print(f'Tweet: {tweets_err[i]}; true label = {gold_err[i]}, prediction = {pred_err[i]}.')

Tweet: $LULU #Outlook #Q1 2013 #lost revenue in the range of $12 million to $17 million -not good; true label = 0, prediction = 2.
Tweet: RBS chairman admits surprise at size of regulatory penalties; true label = 0, prediction = 2.
Tweet: Merged LSE and Deutsche BÃ¶rse would be led by Germany's Kengeter; true label = 1, prediction = 2.
Tweet: Tesco's Sales Pickup Isn't Enough; true label = 0, prediction = 2.
Tweet: $MA http://stks.co/1CYN continues to consolidate / base here; true label = 1, prediction = 2.
Tweet: Aviva shuts Friends Life head office in rapid integration move; true label = 1, prediction = 2.
Tweet: Shell offers 50 percent premium to buy BG for $70 billion; true label = 1, prediction = 2.
Tweet: $ATHN Seems like a good short setup. Stop above the 50 day. No position.; true label = 0, prediction = 2.
Tweet: $KIOR My god. Market getting crushed today. Gotta hold if anything g should recover some tomorrow I imagine.; true label = 0, prediction = 2.
Tweet: $INTC finally som

### Performance on test set

In [21]:
# Let's test the performance on the test set.
y_test_pred = classifier.predict(test_documents)
# Checking performance
print(classification_report(test_labels, y_test_pred))

ValueError: Expected 2D array, got 1D array instead:
array=['$FB turns back down in early trading.... https://t.co/reuf5PzhBm'
 'Whitbread buys 49% stake in Pure food chain'
 "Are ARM Holdings plc, Domino's Pizza Group plc and ASOS plc 3 must-have growth stocks?"
 'CompaniesUnilever sales lifted by ice cream in soft economy'
 'Shell to buy BG Group in $69.7 billion takeover'
 'Industry NewsMorrisons unveils which stores will close as 900 jobs face the axe'
 '$BBRY nice bounce'
 '$SBUX inverted head & shoulders pattern in the works. bullish.'
 'Apple expected to launch new iPhone, iPad today $AAPL https://t.co/XfnICWeBeG'
 'Still short $LNG from $11.70 area...next stop could be down through $9.00. Someone slammed it hard with 230,000 shs this am! More to follow'
 'Infection worries prompt regulatory reviews of Gilead leukaemia drug $GILD https://t.co/H9vqXpZ15G via @YahooFinance'
 'Timberrrr $TSLA https://t.co/MRBgfDHHVa' '$CHRM Loooooongggggg base'
 'Shire CEO steps up drive to get Bax